### 导入 sqlite3模块

In [5]:
from sqlite3 import connect

In [6]:
db_name="test1.db"

### 连接并使用游标

In [7]:
con=connect(db_name)
cur=con.cursor()

### 利用execute进行执行

In [8]:
cur.execute("create table star(id integer,name text,age integer,\
           address text)")

In [9]:
rows=[(1,"王俊凯",16,"重庆"),(2,"王源",15,"重庆"),(3,"易烊千玺",15,"怀化")]

### 占位符的使用

In [10]:
for item in rows:
    cur.execute("insert into star(id,name,age,address)\
    values (?,?,?,?)",item)

In [11]:
cur.execute("select *from star")

In [12]:
for row in cur:
    print(row)

(1, '王俊凯', 16, '重庆')
(2, '王源', 15, '重庆')
(3, '易烊千玺', 15, '怀化')


### 如何查询数据

In [13]:
cur.execute("update star set age=? where id=?",(16,3))

In [14]:
cur.execute("select *from star")
for row in cur:
    print(row)

(1, '王俊凯', 16, '重庆')
(2, '王源', 15, '重庆')
(3, '易烊千玺', 16, '怀化')


### 提交数据

In [15]:
con.commit()

In [16]:
cur.execute("delete from star where id=?",(3,))

In [17]:
cur.execute("select * from star")
for row in cur:
    print(row)

(1, '王俊凯', 16, '重庆')
(2, '王源', 15, '重庆')


In [18]:
con.commit()

### 关闭数据库连接

In [19]:
con.close()

### 行对象


- 支持的操作：
    - 列名进行访问
    - 索引号进行访问
    - 迭代访问
    - len()操作
- 游标建立前：
    - con.row_factory=sqlite3.Row
- 例子如下

In [23]:
from sqlite3 import connect,Row
db_name="test.db"
con=connect(db_name)
con.row_factory=Row
cur=con.cursor()
cur.execute("select * from star")
row=cur.fetchone()
print(type(row))
print("以列名进行访问： ",row["name"])
print("以索引号进行访问： ",row[1])
print("采用迭代形式访问：")
for item in row:
    print(item)

print("len(): ",len(row))
con.close()

<class 'sqlite3.Row'>
以列名进行访问：  王俊凯
以索引号进行访问：  王俊凯
采用迭代形式访问：
1
王俊凯
16
重庆
len():  4


### 批量数据库操作

-  cur.executemany(sql_string,seq)
-  cur.executescript(sql_string)

In [35]:
from sqlite3 import connect,Row
db_name="test.db"
con=connect(db_name)
con.row_factory=Row
cur=con.cursor()

rows=[(5,"Lily",12,"北京"),(6,"John",13,"上海")]
cur.executemany("insert into star (id,name,age,address) values (?,?,?,?)",rows)

cur.execute("select * from star")

for row in cur:
    for r in row:
        print(r)

con.commit()
con.close()

1
王俊凯
16
重庆
2
王源
15
重庆
5
Lily
12
北京
6
John
13
上海


In [39]:
from sqlite3 import  connect
da_name="test1.db"
con=connect(db_name)
cur=con.cursor()
sql_string="""
create table test1(id integer,name text);
insert into test1 (id,name) values(1,"Lily");
insert into test1 (id,name) values(2,"John");
"""
cur.executescript(sql_string)
cur.execute("select * from test1")
for item in cur:
    print(item)
con.commit()
con.close()

(1, 'Lily')
(2, 'John')


### 创建可在SQL语句使用自定的函数

- 创建基本函数

In [41]:
from sqlite3 import connect ,Row
import binascii  #加密的模块

da_name="test1.db"

def encrypt(mydata):
    crc=str(binascii.crc32(mydata.encode()))
    while len(crc)<10:
        crc="0"+crc
    return mydata+crc

def check(mydata):
    if len(mydata)<11:
        return None
    crc_res=str(binascii.crc32(mydata[:-10].encode()))
    while len(crc_res)<10:
        crc_res="0"+crc_res
    if crc_res==mydata[-10:]:
        return mydata[:-10]

In [47]:
con=connect(db_name)
con.create_function("checkk",1,check)
cur=con.cursor()
sql_script="""
drop table if exists testa;
create table if not exists testa(id integer,name text);
insert into testa (id,name) values (3,"%s");
insert into testa (id,name) values (4,"%s");
"""
names=["Lily","Green"]
names=tuple(encrypt(i) for i in names)
sql_script=sql_script % names
print(sql_script)
cur.executescript(sql_script)

cur.execute("update testa set name=? where id=?",("dfddkkjd1122334455",4))
cur.execute("select id,checkk(name) from testa")
for item in cur:
    print(item)
    
con.commit()
con.close()


drop table if exists testa;
create table if not exists testa(id integer,name text);
insert into testa (id,name) values (3,"Lily1460741381");
insert into testa (id,name) values (4,"Green0291225893");

(3, 'Lily')
(4, None)


- 创建聚合函数

In [51]:
from sqlite3 import connect ,Row
import binascii  #加密的模块

da_name="test1.db"

class AbsSum:
        def __init__(self):
            self.s=0
        def step(self,v):
            self.s+=abs(v)
        def finalize(self):
            return self.s

con=connect(db_name)
con.create_aggregate("abssum",1,AbsSum)
cur=con.cursor()

sql_script="""
drop table if exists testa;
create table if not exists testa(id integer,name text,score integer);
insert into testa (id,name,score) values (3,"Lily",90);
insert into testa (id,name,score) values (4,"Green",80);
"""
cur.executescript(sql_script)
cur.execute("select abssum(score) from testa")
for item in cur:
    print(item)
    
con.commit()
con.close()

(170,)


In [1]:
import sqlite3 as lite
import sys

cars = (
    (1, 'Audi', 52642),
    (2, 'Mercedes', 57127),
    (3, 'Skoda', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Hummer', 41400),
    (7, 'Volkswagen', 21600)
)


con = lite.connect('test.db')

with con:
    
    cur = con.cursor()    
    
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)

In [15]:
cur.execute("select * from Cars")
    

In [16]:
import sqlite3 as lite
import sys

con = lite.connect(':memory:')

with con:
    
    cur = con.cursor()    
    cur.execute("CREATE TABLE Friends(Id INTEGER PRIMARY KEY, Name TEXT);")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Tom');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Rebecca');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Jim');")
    cur.execute("INSERT INTO Friends(Name) VALUES ('Robert');")
        
    lid = cur.lastrowid
    print("The last Id of the inserted row is %d" % lid)

The last Id of the inserted row is 4


- 读入或者写出数据

In [19]:
import sqlite3 as lite
import sys

cars = (
    (1, 'Audi', 52643),
    (2, 'Mercedes', 57642),
    (3, 'Skoda', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Hummer', 41400),
    (7, 'Volkswagen', 21600)
)

def writeData(data):
    
    f = open('cars.sql', 'w')
    
    with f:
        f.write(data)


con = lite.connect(':memory:')

with con:
    
    cur = con.cursor()
    
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)
    cur.execute("DELETE FROM Cars WHERE Price < 30000")
    
    data = '\n'.join(con.iterdump())
    
    writeData(data)

In [20]:
import sqlite3 as lite
import sys


def readData():
    
    f = open('cars.sql', 'r')
    
    with f:
        data = f.read()
        return data
        

con = lite.connect(':memory:')

with con:   

    cur = con.cursor()
    
    sql = readData()
    cur.executescript(sql)
    
    cur.execute("SELECT * FROM Cars")
    
    rows = cur.fetchall()
    
    for row in rows:
        print(row)    

(1, 'Audi', 52643)
(2, 'Mercedes', 57642)
(5, 'Bentley', 350000)
(6, 'Hummer', 41400)
